# Simple loop agent

In this example, we will create an loop agent that simply generate a description base on a same query and get all results

In other word, this is a "for loop" agent.

In [1]:
from collections.abc import Sequence
from typing import List

from a2a.types import AgentCapabilities, AgentCard, AgentSkill
from aap_core.agent import BaseAgent
from aap_core.chain import BaseLLMChain
from aap_core.orchestration import LoopAgent
from aap_core.types import AgentMessage, AgentResponse
from aap_dspy.chain import BaseSignatureAdapter, ChatCausalMultiTurnsChain

import dspy

In [2]:
class QA(dspy.Signature):
    guide: str = dspy.InputField(optional=True)
    question: str = dspy.InputField()
    answer: str = dspy.OutputField()
    history: dspy.History = dspy.InputField()


class SignatureAdapter(BaseSignatureAdapter[QA]):
    def msg2sig(self, message: AgentMessage) -> List[QA]:
        history = dspy.History(messages=[])
        q, a = None, None
        for response in message.responses:
            if response[0] == "user":
                q = response[1]
            elif response[0] == "tool":
                pass
            elif response[0] == "system":
                pass
            else:
                a = response[1]

            if q is not None and a is not None:
                history.messages.append({"question": q, "answer": a})
                q, a = None, None

        guide_str = self._prefill_dict.get("guide", "")
        qa = QA(question=message.query, answer="", history=history, guide=guide_str)
        return [qa]

    def sig2msg(self, signatures: List[QA], name: str) -> List[AgentResponse]:
        responses = []
        for signature in signatures:
            responses.append(("user", signature.question))
            responses.append((name, signature.answer))
        return responses

In [3]:
class Agent(BaseAgent):
    chain: BaseLLMChain

    def execute(self, message: AgentMessage, **kwargs) -> AgentMessage:
        self.state = "running"
        message = self.chain.invoke(message, **kwargs)
        message.execution_result = "success"
        message.origin = self.card.name
        self.state = "idle"
        return message

In [4]:
from collections.abc import Generator

system_prompt = """You are a genius and creative writer."""

max_iteration = 5

def state_callback(state: str):
    print(f"agent state: {state}")

def iter(max_iteration: int) -> Generator[bool, None, None]:
    n = 0
    while True:
        yield n >= max_iteration
        n = (n + 1) % (max_iteration + 1)

model = dspy.LM(model="ollama_chat/llama3.2:latest", api_base="http://192.168.55.1:11434", api_key="", temperature=1.5, cache=False)
adapter = SignatureAdapter.with_prefill({"guide": system_prompt})
predictor = dspy.ChainOfThought(QA)
chain = ChatCausalMultiTurnsChain(QA, predictor=predictor, adapter=adapter).with_lm(model)
writer_skill = AgentSkill(
    id='writer-skill',
    name="writer skill",
    description="writer skill",
    tags=['writer']
)
writer_card = AgentCard(
    name="writer agent",
    description="Writer agent",
    skills=[writer_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
loop_skill = AgentSkill(
    id='loop-skill',
    name="loop skill",
    description="loop skill",
    tags=['loop']
)
loop_card = AgentCard(
    name="loop agent",
    description="loop agent",
    skills=[loop_skill],
    capabilities=AgentCapabilities(),
    default_input_modes=['text'],
    default_output_modes=['text'],
    url="localhost",
    version="0.1.0"
)
agent = Agent(chain=chain, card=writer_card, state_change_callback=state_callback)
loop_agent = LoopAgent(agent=agent, is_stop=iter(max_iteration), card=loop_card, state_change_callback=state_callback)

In [5]:
query = "Write a short poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=-1)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: loop agent:running/writer agent:idle
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#0/writer agent:running
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#1/writer agent:running
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#2/writer agent:running
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#3/writer agent:running
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:running#4/writer agent:running
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:idle/writer agent:idle
query="Write a short poem with Shakespere's style about the peace before the storm." query_media=None origin

In [6]:
query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=1)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: loop agent:running/writer agent:idle
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#0/writer agent:running
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#1/writer agent:running
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#2/writer agent:running
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#3/writer agent:running
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:running#4/writer agent:running
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:idle/writer agent:idle
query="Write a poem with Shakespere's style about the peace before the storm." query_media=None origin='loop

In [7]:
query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=3)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: loop agent:running/writer agent:idle
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#0/writer agent:running
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#1/writer agent:running
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#2/writer agent:running
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#3/writer agent:running
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:running#4/writer agent:running
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:idle/writer agent:idle
query="Write a poem with Shakespere's style about the peace before the storm." query_media=None origin='loop

In [8]:
def custom_keep_logic(responses: Sequence[AgentResponse]) -> List[AgentResponse]:
    return list(responses)


query = "Write a poem with Shakespere's style about the peace before the storm."
message = loop_agent.execute(AgentMessage(query=query), keep_result=custom_keep_logic)

print(message)
print(f"message responses len: {len(message.responses)}")

for agent_name, response in message.responses:
    print(f"{agent_name}: {response}")
    print("-" * 50)
    print()

agent state: loop agent:running/writer agent:idle
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#0/writer agent:running
agent state: loop agent:running#0/writer agent:idle
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#1/writer agent:running
agent state: loop agent:running#1/writer agent:idle
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#2/writer agent:running
agent state: loop agent:running#2/writer agent:idle
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#3/writer agent:running
agent state: loop agent:running#3/writer agent:idle
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:running#4/writer agent:running
agent state: loop agent:running#4/writer agent:idle
agent state: loop agent:idle/writer agent:idle
query="Write a poem with Shakespere's style about the peace before the storm." query_media=None origin='loop

In [9]:
dspy.inspect_history(n=10)





[2025-12-10T12:26:09.166806]

System message:

Your input fields are:
1. `guide` (str): 
2. `question` (str): 
3. `history` (History):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## guide ## ]]
{guide}

[[ ## question ## ]]
{question}

[[ ## history ## ]]
{history}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `guide`, `question`, `history`, produce the fields `answer`.


User message:

[[ ## guide ## ]]
You are a genius and creative writer.

[[ ## question ## ]]
Write a poem with Shakespere's style about the peace before the storm.

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## reasoning ## ]